In [0]:
output_path="dbfs:/FileStore/files"

In [0]:
import requests
import json

# Lista para almacenar los datos de cada usuario
usuarios = []
# Hacer 5 peticiones a la API
for i in range(5):
    response = requests.get("https://randomuser.me/api/")
    if response.status_code == 200:
        data = response.json()
        usuarios.append(data['results'][0])  # Guardamos solo la info del usuario
    else:
        print(f"Error en la petición {i+1}: {response.status_code}")

# Guardar los datos en un archivo JSON
with open(f"{output_path}/usuarios/usuarios.json", "w", encoding="utf-8") as f:
    json.dump(usuarios, f, indent=2, ensure_ascii=False)

print("✅ Archivo 'usuarios.json' creado con éxito.")

In [0]:
# Leer el archivo JSON (ubicado en raíz tras la carga)
df_usuarios = spark.read.option("multiline", "true").json(f"{output_path}/usuarios/usuarios.json")
# Muestra el esquema del DataFrame
df_usuarios.printSchema()
df_contactos = df_usuarios.selectExpr(
    "id.value as _id",
    "email as correo",
    "gender as genero",
    "login.username as usuario",
    "name.first as nombre",
    "name.last as apellido",
    "phone as telefono"
    )
df_contactos.show()
# Escribe como TSV
df_contactos.write\
      .mode("overwrite")\
      .option("header", True)\
      .option("delimiter", "|")\
      .option("compression", "gzip")\
      .csv(f"{output_path}/contactos_flat")
print("Archivo plano generado")
# Escribe como parquet
df_contactos.write\
      .mode("overwrite")\
      .save(f"{output_path}/contactos_parquet")
print("Archivo parquet generado")
# Lectura de parquet
df = spark.read.parquet(f"{output_path}/contactos_parquet")
df.show(3)